# 1. DATA

In [1]:
# Importing ibraries
import pandas as pd
import datetime
import numpy as np
import string
import re
import nltk
from nltk import sent_tokenize, word_tokenize , WordPunctTokenizer
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edudi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
url = 'BD_HackMTY_ChoquesYSiniestros_2020_V2.xlsx'
df = pd.read_excel(url, encoding='utf-8', na_values=['NA',r'\N','0'], converters={'DATE':str})

In [3]:
# As dates are inversed from US format we need to overwrite the DATE column
df.dropna(how='any', subset=['AÑO'], inplace=True)
df.rename(columns={'AÑO':'year','MES':'month','DÍA NUMERO':'day'}, inplace=True)
df.DATE = pd.to_datetime(df[['year','month','day']])
df.rename(columns={'year':'AÑO','month':'MES','day':'DÍA NUMERO'}, inplace=True)

# 2. Data Wrangling

In [4]:
# Changing Day number to ordinal values
mapping= {'LUNES':1,
          'MARTES':2,
          'MIERCOLES':3,
          'JUEVES':4,
          'VIERNES':5,
          'SABADO':6,
          'DOMINGO':7}
#Replace values and collapse data
df.DIA = df.DIA.replace(mapping)


In [7]:
def nulls(df):
    tot = df.isnull().sum().sort_values(ascending = False)
    per = df.isnull().sum()/df.isnull().count().sort_values(ascending = False)
    missing_cont = pd.concat([tot, per] , axis = 1 , keys = ['Total', 'Percentage'])
    print(missing_cont)

In [8]:
def uniques(df):
    print("Unique Value Count:")
    cols = df.columns.tolist()
    for col in cols:
        print(col + " = " + str(len(df[col].unique())))


In [9]:
uniques(df)                                                                

2012.0    9802
2011.0    9193
2015.0    8658
2013.0    7011
2014.0    6997
2008.0    6770
2007.0    6760
2016.0    6208
2006.0    5556
2005.0    5288
2009.0    5153
2017.0    4741
2010.0    4490
2004.0    3756
2003.0    3322
NaN       2924
2002.0    2890
2001.0    2832
2000.0    2517
1999.0    1718
1998.0    1540
1997.0    1035
1989.0     744
1995.0     714
1990.0     658
1994.0     657
1992.0     649
1996.0     614
1993.0     585
2018.0     555
1000.0     486
1991.0     437
1988.0     322
1983.0      56
1987.0      39
1986.0      38
1980.0      33
1981.0      29
1950.0      24
1984.0      22
1978.0      21
1985.0      20
1979.0      18
1973.0      17
1982.0      17
1977.0      15
1975.0      15
1976.0      12
1974.0      10
1954.0       5
1965.0       4
1952.0       4
1972.0       4
1970.0       4
1966.0       3
1969.0       3
1961.0       2
1967.0       1
1962.0       1
1955.0       1
1951.0       1
1971.0       1
2071.0       1
2019.0       1
Name: MODELO_VEHICULO, dtype: int64

In [10]:
def bucket_model(variable):
    if variable >= 2015:
        return '2015-2020'
    elif (variable >= 2010 and variable < 2015) :
         return '2010-2015'
    elif (variable >= 2005 and variable < 2010) : 
        return '2005-2010'
    elif (variable >= 2000 and variable < 2005) : 
        return '2000-2005'
    elif variable < 2000 :
        return 'older'

In [11]:
df['MODELO_VEHICULO'] = df['MODELO_VEHICULO'].astype(str).astype(float)
df['MODEL_YEAR'] = df['MODELO_VEHICULO'].apply(bucket_model)


In [12]:
df['MODEL_YEAR'].value_counts(dropna = False)

2010-2015    37493
2005-2010    29527
2015-2020    20164
2000-2005    15317
older        10579
NaN           2924
Name: MODEL_YEAR, dtype: int64

## Car Color Tokenization

In [13]:
pd.options.display.max_rows = 200
df['COLOR'].value_counts(dropna = False)[:100]

BLANCO             32366
GRIS               18035
ROJO                9817
PLATA               7356
NEGRO               6774
AZUL                6430
ARENA               3411
VERDE               2731
BLANCA              2382
blanco              1953
AMARILLO            1892
NaN                 1354
GUINDA              1279
gris                1144
VINO                1134
DORADO               809
CAFE                 766
GRIS PLATA           707
rojo                 561
NEGRA                524
GUINDO               496
ROJA                 448
plata                425
azul                 412
negro                382
Blanco               356
NARANJA              355
BCO                  245
BEIGE                238
arena                217
ANARANJADO           214
blanca               208
Gris                 200
verde                196
CELESTE              160
BLANCO CON           158
MORADO               155
CREMA                149
ORO                  147
BLANCO/ROJO          139


In [14]:
def clean_column(text):
    #remove digits 
    text_nonum = re.sub(r'\d+', '', text)
    #convert to lower case
    text_nopunct = ''.join([char.lower() for char in text_nonum if char not in string.punctuation]) #remove special characters
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace


In [15]:
df_new = df.copy()
df_new.COLOR = df_new.COLOR.apply(str)
df_new.COLOR = df_new.COLOR.str.lower()
df_new.COLOR = df_new.COLOR.apply(clean_column)
df_new.COLOR = df_new.COLOR.apply(word_tokenize)

In [16]:
import difflib
def color_match(text):
    ls= ['blanco','gris','rojo','plata','negro','azul','arena',
         'verde','amarillo','guindo','vino','dorado',
         'cafe','naranja','morado','crema']
    for element in text:
        if element in ls:
            return element
        elif element == 'nan':
            return np.nan
        elif element == 'celeste':
            return 'azul'
        elif element == 'beige':
            return 'arena'
        elif element == 'champagne':
            return 'cafe'
        else:
            similar = difflib.get_close_matches(element, ls)
            if similar != []:
                return similar[0]


In [17]:
df2 = df_new.copy()
df2.COLOR = df2.COLOR.apply(color_match)
mapping = {'guindo':'rojo','crema':'cafe','vino':'rojo','arena':'cafe','gris':'plata'}
df2.COLOR = df2.COLOR.replace(mapping)

In [18]:
df2.COLOR.value_counts(dropna = False)

blanco      40569
plata       29395
rojo        14809
negro        8102
azul         7919
cafe         5232
verde        3285
amarillo     2291
NaN          2128
dorado        996
naranja       669
morado        609
Name: COLOR, dtype: int64

In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116004 entries, 0 to 116010
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   FOLIO_ID             116004 non-null  object        
 1   LAT                  104246 non-null  float64       
 2   LONG                 104246 non-null  float64       
 3   CODIGO POSTAL        104223 non-null  object        
 4   CALLE                115989 non-null  object        
 5   COLONIA              113321 non-null  object        
 6   CAUSA SINIESTRO      116004 non-null  object        
 7   TIPO VEHICULO        115507 non-null  object        
 8   COLOR                113876 non-null  object        
 9   MODELO_VEHICULO      113080 non-null  float64       
 10  NIVEL DAÑO VEHICULO  107033 non-null  object        
 11  PUNTO DE IMPACTO     104526 non-null  object        
 12  AÑO                  116004 non-null  float64       
 13  MES           

In [20]:
def clean_LAT_LON(data):
    # Create a copy of the dataframe
    df = data.copy()
    # Change all the latitudes and latitudes that does not correspond of NL to NaN
    df['LAT'][(df['LAT'] > 27.78) | (df['LAT'] < 23.16)] = np.nan
    df['LONG'][(df['LONG'] > -98.42) | (df['LONG'] < -101.20)] = np.nan
    # Quitar aquellas columnas con NaN values
    df = df.dropna(how='any', subset=['LAT','LONG'])

    return df

In [21]:
df3 = clean_LAT_LON(df2)

In [22]:
nulls(df3)

Total  Percentage
PUNTO DE IMPACTO      9613    0.092303
NIVEL DAÑO VEHICULO   7601    0.072984
MODELO_VEHICULO       2562    0.024600
MODEL_YEAR            2562    0.024600
COLOR                 1769    0.016986
COLONIA               1733    0.016640
HORA                   949    0.009112
TIPO VEHICULO          444    0.004263
CIUDAD_APROXIMADA      251    0.002410
CODIGO POSTAL          162    0.001556
CALLE                    4    0.000038
MES                      0    0.000000
AÑO                      0    0.000000
ESTADO                   0    0.000000
DATE                     0    0.000000
DIA                      0    0.000000
CAUSA SINIESTRO          0    0.000000
DÍA NUMERO               0    0.000000
LONG                     0    0.000000
LAT                      0    0.000000
FOLIO_ID                 0    0.000000


In [23]:
uniques(df3)

Unique Value Count:
FOLIO_ID = 104146
LAT = 84745
LONG = 85421
CODIGO POSTAL = 749
CALLE = 41573
COLONIA = 17755
CAUSA SINIESTRO = 7
TIPO VEHICULO = 5
COLOR = 13
MODELO_VEHICULO = 63
NIVEL DAÑO VEHICULO = 5
PUNTO DE IMPACTO = 13
AÑO = 3
MES = 12
DÍA NUMERO = 31
DIA = 7
HORA = 24
ESTADO = 1
CIUDAD_APROXIMADA = 96
DATE = 911
MODEL_YEAR = 6


In [24]:
df4 = df3.drop(['CODIGO POSTAL','CALLE','COLONIA','MODELO_VEHICULO'],axis = 1)

In [25]:
df4['PUNTO DE IMPACTO'].value_counts(dropna = False)

Frontal                  23621
Trasero                  23561
Cofre                    13782
NaN                       9613
Costado der delantero     7770
Costado izq delantero     6377
Costado der trasero       5344
Costado izq trasero       5084
Costado der central       3841
Costado izq central       3501
Parte baja                 915
Toldo                      676
Tapa cajuela                61
Name: PUNTO DE IMPACTO, dtype: int64

In [26]:
df4['PUNTO DE IMPACTO'].fillna(df4['PUNTO DE IMPACTO'].mode()[0], inplace =True)
df4['NIVEL DAÑO VEHICULO'].fillna(df4['NIVEL DAÑO VEHICULO'].mode()[0], inplace =True)
df4['COLOR'].fillna(df4['COLOR'].mode()[0], inplace =True)
df4['TIPO VEHICULO'].fillna(df4['TIPO VEHICULO'].mode()[0], inplace =True)
df4['CIUDAD_APROXIMADA'].fillna(df4['CIUDAD_APROXIMADA'].mode()[0], inplace =True)
df4['ESTADO'].fillna(df4['ESTADO'].mode()[0], inplace =True)
df4['MODEL_YEAR'].fillna(df4['MODEL_YEAR'].mode()[0], inplace =True)
df4['CAUSA SINIESTRO'].fillna(df4['CAUSA SINIESTRO'].mode()[0], inplace =True)
df4['HORA'].fillna(df4['HORA'].median(), inplace = True)
#df4['AÑO'].fillna(df4['AÑO'].median(), inplace = True)
df4['DIA'].fillna(df4['DIA'].median(), inplace = True)
#df4['DÍA NUMERO'].fillna(df4['DÍA NUMERO'].median(), inplace = True)
#df4['MES'].fillna(df4['MES'].median(), inplace = True)
#df4['DATE'].fillna(df4['DATE'].mode(), inplace = True)

In [27]:
nulls(df4)

Total  Percentage
MODEL_YEAR               0         0.0
PUNTO DE IMPACTO         0         0.0
LAT                      0         0.0
LONG                     0         0.0
CAUSA SINIESTRO          0         0.0
TIPO VEHICULO            0         0.0
COLOR                    0         0.0
NIVEL DAÑO VEHICULO      0         0.0
AÑO                      0         0.0
DATE                     0         0.0
MES                      0         0.0
DÍA NUMERO               0         0.0
DIA                      0         0.0
HORA                     0         0.0
ESTADO                   0         0.0
CIUDAD_APROXIMADA        0         0.0
FOLIO_ID                 0         0.0


In [28]:
uniques(df4)

Unique Value Count:
FOLIO_ID = 104146
LAT = 84745
LONG = 85421
CAUSA SINIESTRO = 7
TIPO VEHICULO = 4
COLOR = 11
NIVEL DAÑO VEHICULO = 4
PUNTO DE IMPACTO = 12
AÑO = 3
MES = 12
DÍA NUMERO = 31
DIA = 7
HORA = 23
ESTADO = 1
CIUDAD_APROXIMADA = 95
DATE = 911
MODEL_YEAR = 5


In [29]:
# The working directory is setted to save the file
import os
os.chdir('Weather_Info')

In [30]:
pip install wwo_hist

Note: you may need to restart the kernel to use updated packages.


In [31]:
# Import the library
from wwo_hist import retrieve_hist_data
import datetime

In [32]:
'''frequency = 1
start_date = '01-JAN-2016'
end_date = '31-DEC-2018'
api_key = 'ac6824961322437795f10424203008'
location_list = ['monterrey,mexico']
hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)'''

"frequency = 1\nstart_date = '01-JAN-2016'\nend_date = '31-DEC-2018'\napi_key = 'ac6824961322437795f10424203008'\nlocation_list = ['monterrey,mexico']\nhist_weather_data = retrieve_hist_data(api_key,\n                                location_list,\n                                start_date,\n                                end_date,\n                                frequency,\n                                location_label = False,\n                                export_csv = True,\n                                store_df = True)"

In [33]:
weather_url = 'monterrey,mexico.csv'
df_weather = pd.read_csv(weather_url)

In [34]:
df_weather.date_time  = pd.to_datetime(df_weather.date_time)

In [35]:
def calculate_date_time(data):
    df = data.copy()
    df = df.dropna(how='any', subset=['DATE'])
    df.reset_index(drop=True, inplace=True)
    # Create column
    df['complete_date'] = datetime.datetime.today()
    for element in range(len(df)):
        # Save time
        time = datetime.time(int(df.HORA[element]), 0)
        # Combine both dates and save them
        df['complete_date'][element] = datetime.datetime.combine(df.DATE[element], time)
    # Eliminate unused columns
    df.drop(['HORA','DATE'], axis=1, inplace=True)

    return df

In [36]:
df5 = calculate_date_time(df4)

In [37]:
df5.drop_duplicates(keep=False, inplace=True)

In [38]:
df_collisions = pd.merge(df5,df_weather, how='left', left_on='complete_date', right_on='date_time')

In [39]:
import datetime

def holidays(x):
    import holidays
    mex_holidays = holidays.MX()
    if datetime.date(int(x['AÑO']),int(x['MES']),int(x['DÍA NUMERO'])) in mex_holidays:
        return 1
    else:
        return 0
df_collisions['holiday'] = df_collisions.apply(holidays, axis = 1)
df_collisions.holiday

0         0
1         0
2         0
3         0
4         0
         ..
104141    0
104142    1
104143    0
104144    0
104145    0
Name: holiday, Length: 104146, dtype: int64

In [40]:
df_collisions

,FOLIO_ID,LAT,LONG,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,holiday
0,D00002,25.647351,-100.140132,COLISION Y/O VUELCO,Camión,blanco,Sin daño,Costado izq trasero,2018.0,1.0,...,10,27,64,0.1,1027,8,9,112,8,0
1,D00004,25.718027,-100.249990,COLISION Y/O VUELCO,Auto,plata,Sin daño,Frontal,2018.0,1.0,...,3,5,61,0.0,1024,7,10,295,3,0
2,D00007,25.660916,-100.282504,COLISION Y/O VUELCO,Auto,plata,Bajo,Frontal,2018.0,1.0,...,7,4,24,0.0,1019,17,10,28,6,0
3,D00008,25.723590,-100.198780,COLISION Y/O VUELCO,Auto,blanco,Sin daño,Frontal,2018.0,1.0,...,16,5,41,0.0,1028,14,10,84,11,0
4,D00009,25.708889,-100.246876,COLISION Y/O VUELCO,Auto,amarillo,Bajo,Frontal,2018.0,2.0,...,5,9,84,0.0,1017,15,10,76,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104141,D116010,25.659807,-100.146937,COLISION Y/O VUELCO,Auto,plata,Bajo,Cofre,2016.0,1.0,...,15,0,29,0.0,1022,16,10,91,13,0
104142,D116011,24.922858,-99.688721,COLISION Y/O VUELCO,Camión,blanco,Medio,Cofre,2016.0,12.0,...,6,0,82,0.0,1014,15,10,122,4,1
104143,D116012,25.745930,-100.360152,COLISION Y/O VUELCO,Auto,rojo,Sin daño,Costado der trasero,2016.0,12.0,...,16,90,68,0.0,1028,12,8,251,13,0
104144,D116013,25.658918,-100.371243,COLISION Y/O VUELCO,Auto,azul,Sin daño,Cofre,2016.0,1.0,...,2,8,11,0.0,1011,25,10,131,2,0


In [41]:
df_collisions = df_collisions[df_collisions['CAUSA SINIESTRO'] == 'COLISION Y/O VUELCO']

In [43]:
#We only want the crashes
df5 = df5[df5['CAUSA SINIESTRO'] == 'COLISION Y/O VUELCO']

In [42]:
#Export for Classification
#df_collisions.to_csv('df_collisions_id.csv')

In [44]:
# Group the collision dataframe to get the count of cases in a day and merge it with weather data
time_series = pd.merge(df_weather, df5.groupby('complete_date', as_index=False)['FOLIO_ID'].agg('count').rename(columns={'FOLIO_ID':'NUM_COLLISIONS'}), how='left', left_on='date_time', right_on='complete_date')
time_series.drop(['complete_date'], axis=1, inplace=True)
# Replace all missing days with 0 to indicate no collision
time_series['NUM_COLLISIONS'].fillna(0, inplace=True)
# Convert to int type
time_series['NUM_COLLISIONS'] = time_series['NUM_COLLISIONS'].astype(int)

In [45]:
# Additional information for the function of holiday to work
time_series['AÑO'] = time_series.date_time.dt.year
time_series['MES'] = time_series.date_time.dt.month
time_series['DÍA NUMERO'] = time_series.date_time.dt.day

In [46]:
# Apply the holiday function to recognize if that date is a holiday
time_series['holiday'] = time_series.apply(holidays, axis = 1)
time_series.drop(['AÑO','MES','DÍA NUMERO'], axis=1, inplace=True)

In [47]:
# Change the order of the las column to be the target variable
time_series = time_series[[c for c in time_series if c not in ['NUM_COLLISIONS']]+['NUM_COLLISIONS']]

In [48]:
time_series

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,moonset,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,holiday,NUM_COLLISIONS
0,2016-01-01 00:00:00,10,5,0.0,5.4,3,1,50,12:59 AM,01:17 PM,...,54,65,0.0,1025,9,10,70,5,1,0
1,2016-01-01 01:00:00,10,5,0.0,5.4,3,1,50,12:59 AM,01:17 PM,...,65,66,0.0,1024,9,10,71,5,1,5
2,2016-01-01 02:00:00,10,5,0.0,5.4,3,1,50,12:59 AM,01:17 PM,...,77,67,0.0,1024,9,10,73,5,1,2
3,2016-01-01 03:00:00,10,5,0.0,5.4,3,1,50,12:59 AM,01:17 PM,...,89,68,0.0,1024,8,10,74,5,1,2
4,2016-01-01 04:00:00,10,5,0.0,5.4,3,1,50,12:59 AM,01:17 PM,...,79,68,0.0,1024,8,10,64,6,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27043,2018-12-31 19:00:00,16,5,0.0,6.9,3,1,36,03:32 AM,03:19 PM,...,24,51,0.0,1015,12,10,120,5,0,0
27044,2018-12-31 20:00:00,16,5,0.0,6.9,3,1,36,03:32 AM,03:19 PM,...,30,56,0.0,1015,12,10,106,4,0,0
27045,2018-12-31 21:00:00,16,5,0.0,6.9,3,1,36,03:32 AM,03:19 PM,...,36,62,0.0,1016,13,10,92,3,0,0
27046,2018-12-31 22:00:00,16,5,0.0,6.9,3,1,36,03:32 AM,03:19 PM,...,24,63,0.0,1016,11,10,91,2,0,0


In [49]:
#Export to explore the time_series
# time_series.to_csv('time_series_data.csv')